# Les APIs REST avec Python

## Création API REST avec Flask

### Installation

In [ ]:
# Installation avec Pip (PyPI)
#!pip install -U Flask

# Installation avec Anaconda (Conda)
#!conda install -c anaconda flask

Exemple minimaliste

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Connexion réussie !"

app.run()

### Configuration de Flask
*Différentes manières de configurer, **paramètres basiques** pour l'instant*

Configuration directe

In [ ]:
# Méthode 1: Configuration directe
app.config['DEBUG'] = True
app.config['SECRET_KEY'] = 'un_secret_très_sécurisé'

Configurations améliorées pour davantage de praticité et de sécurité

In [ ]:
# Méthode 2: Configuration à partir d'une classe séparée
class Config(object):
    DEBUG = True
    SECRET_KEY = 'un_secret_très_sécurisé'

app.config.from_object(Config)

# Méthode 3: Configuration à partir d'un fichier CFG
app.config.from_pyfile('config/config.cfg')

# Méthode 4: Configuration à partir de variables d'environnement
import os
app.config['DEBUG'] = os.environ.get('DEBUG', False)
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY', 'un_secret_très_sécurisé')

# Méthode 5: Configuration à partir d'un fichier JSON
import json
with open('config/config.json') as f:
    app.config.update(json.load(f))

### Définition des routes
Ajout de routes basiques

In [ ]:
# Route basique
@app.route('/login')
def login():
    return 'login'

# Une fonction peut être associée à plusieurs routes.
@app.route('/hello')
@app.route('/bonjour')
def hello():
    return 'Bonjour!'

Ajout de routes avec paramètres

In [ ]:
# Route avec paramètre simple
@app.route('/user/<username>')
def profile(username):
    return f'{username}\'s profile'

# Route avec paramètre converti / casté (en int)
@app.route('/post/<int:post_id>')
def show_post(post_id):
    return f'Post {post_id}'

# Passage de plusieurs paramètres
@app.route('/user/<username>/<int:age>')
def user(username, age):
    return f'{username} is {age} years old'

# Passage de paramètres optionnels
@app.route('/user/<username>/<int:age>')
@app.route('/user/<username>')
def user(username, age=None):
    if age:
        return f'{username} is {age} years old'
    else:
        return f'{username} is ageless'

# Passage de paramètres optionnels avec valeur par défaut [option 1]
# Valeur par défaut inscrit dans la fonction
@app.route('/user/<username>/<int:age>')
@app.route('/user/<username>')
@app.route('/user/<username>/<int:age>/<int:height>')
def user(username, age=None, height=180):
    if age:
        return f'{username} is {age} years old and {height} cm tall'
    else:
        return f'{username} is ageless'
    
# Passage de paramètres optionnels avec valeur par défaut [option 2]
# Valeur par défaut inscrit dans la route
@app.route('/blog', defaults={'page': 1})
@app.route('/blog/page/<int:page>')
def blog(page):
    return f'Page du blog {page}'

### Définition des méthodes CRUD

In [ ]:
# GET (par défaut)
@app.route('/get_example')
def get_example():
    return "Ceci est une réponse de GET!"

# POST
from flask import request
@app.route('/post_example', methods=['POST'])
def post_example():
    # Traitement des données envoyées (données brutes)
    return request.args.get('keyword')  # TODO CHECK

# GET et POST (classique pour un formulaire)
@app.route('/multi_method', methods=['GET', 'POST'])
def multi_method():
    if request.method == 'POST':
        # Traitement des données envoyées (données de formulaire)
        return request.form['name']
    else:
        # GET par défaut
        return "Renvoi du formulaire"

# PUT
@app.route('/update_item', methods=['PUT'])
def update_item():
    # Code pour mettre à jour un élément
    return "Élément mis à jour"

# DELETE
@app.route('/delete_item', methods=['DELETE'])
def delete_item():
    # Code pour supprimer un élément
    return "Élément supprimé"


### Définition des réponses

In [ ]:
# Renvoi d'une réponse simple
@app.route('/')
def home():
    return "Bienvenue sur ma page d'accueil!"

# Utilisation de make_response pratique pour ajouter 
# des éléments (code, headers, etc.) à une réponse pré-existante 
from flask import make_response
@app.route('/custom')
def custom_response():
    resp = make_response("Contenu personnalisé", 202)
    resp.headers['X-Something'] = 'Valeur'
    return resp

# Renvoi d'un objet Response avec la classe Response
# Permet un contrôle direct et complet sur les données
# de réponse, statut HTTP, en-têtes, cookies, etc.
# TODO CREER PLUSIEURS VARIATIONS DE REPONSES
from flask import Response
@app.route('/response')
def response_example():
    return Response("Contenu complet", status=200, headers={'Content-Type': 'text/plain'})

# Renvoi d'un JSON
from flask import jsonify
@app.route('/json')
def json_example():
    return jsonify({'key': 'value', 'listKey': [1, 2, 3]})

# Streaming
from flask import stream_with_context
@app.route('/stream')
def stream_example():
    def generate():
        yield 'Hello '
        yield 'World!'
    return Response(stream_with_context(generate()))

# Redirection vers une autre page
from flask import redirect
@app.route('/redirect')
def redirect_example():
    return redirect('/')

# Redirection vers une autre page avec url_for
# Permet de ne pas avoir à écrire l'URL en dur
# Nécessite d'avoir défini les routes avec un nom
# Exemple: app.route('/', name='home')
from flask import url_for
@app.route('/redirect')
def redirect_example():
    return redirect(url_for('home'))

### Lancer le serveur (API) 

Lancer cette commande dans votre terminal, ou dans un autre notebook
```bash
flask --app example_1_stream.py run --debug
```

In [4]:
# Stream call exemple
import requests

with requests.get("http://localhost:5000/stream", stream=True) as response:
    # chunk_size à None pour que la taille des chuncks soit définie par le serveur automatiquement
    for chunk in response.iter_content(chunk_size=None):  
        print(chunk.decode()) 

Hello 
World!
This
is
a
stream
example.


### Gestion d'erreurs

Gestionnaires d'erreurs personnalisés

In [ ]:
@app.errorhandler(404)
def not_found_error(error):
    return jsonify({'error': 'Resource not found'}), 404

@app.errorhandler(500)
def internal_error(error):
    return jsonify({'error': 'Internal server error'}), 500

Gestion des exceptions (erreurs **de code**)

In [ ]:
@app.errorhandler(ValueError)
def handle_value_error(error):
    return jsonify({'error': str(error)}), 400

Gestion d'erreurs globales
- Uniformité : Permet d'avoir un traitement uniforme des erreurs spécifiques à travers toute l'application.
- Maintenance : Centralise la gestion d'erreurs spécifiques, rendant le code plus facile à maintenir et à mettre à jour.
- Simplicité : Évite de répéter la même logique de gestion d'erreurs dans différentes parties de l'application.

In [ ]:
def global_error_handler(error):
    return jsonify({'error': 'Something went wrong'}), 500

app.register_error_handler(500, global_error_handler)

Création d'une erreur sous condition enfreinte

In [ ]:
from flask import abort

@app.route('/api')
def my_api():
    # Condition d'erreur
    if error_condition:
        abort(400, description='Invalid request')

Logging des erreurs

In [ ]:
import logging

@app.errorhandler(500)
def error_500_handler(error):
    app.logger.error(f'Internal Server Error: {error}')
    return jsonify({'error': 'Internal server error'}), 500

## Consommation des APIs REST avec Python (**coté client**)

### Utilisation de la bibliothèque requests
Lancez le serveur dans votre terminal

```bash
flask --app example_3_basic_api_items.py run --debug
```

In [7]:
# POST
item_name = 'carotte'
price = 2
new_item = requests.post(f'http://localhost:5000/item/{item_name}', json={'price': price})
print(new_item)
print(new_item.json())

<Response [201]>
{'data': {'price': 2}, 'name': 'carotte'}


In [8]:
# GET
item_1 = requests.get(f'http://localhost:5000/item/{item_name}')
print(item_1)
try:
    print(item_1.json())
except:
    print('not found')

<Response [200]>
{'data': {'price': 2}, 'name': 'carotte'}


In [9]:
# PUT
import numpy as np
new_price = np.random.randint(3, 30)

update_item = requests.put(f'http://localhost:5000/item/{item_name}', json={item_name: new_price})
print(update_item)
print(update_item.json())

<Response [200]>
{'data': {'carotte': 21}, 'name': 'carotte'}


In [10]:
# DELETE
delete_item = requests.delete(f'http://localhost:5000/item/{item_name}')
print(delete_item)
print(delete_item.json())

<Response [200]>
{'message': 'Item deleted'}


In [11]:
# GET sur tous les items
items = requests.get('http://localhost:5000/items')
print(items)
print(*items.json())
print(items.json())

<Response [200]>
poireau
{'poireau': {'data': {'poireau': 3}, 'name': 'poireau'}}


### Gestion des réponses

- Interprétation du statut HTTP
- Extraction, utilisation des données
- Gestion des erreurs et exceptions (et extraction des messages d'erreurs transmis par l'API)
- Logging des réponses

Interprétation du statut HTTP

In [12]:
# GET et analyse du statut
fonctionne = 'http://localhost:5000/items'
ne_fonctionne_pas = 'http://localhost:5000/item/fruit_inconnu'

item_1 = requests.get(fonctionne)

if item_1.status_code == 200:
    print("Tout fonctionne bien ! Status Code:", item_1.status_code)
    print(item_1.json())
    # Traiter la réponse ici, si nécessaire
else:
    print("Problème détecté ! Status Code:", item_1.status_code)
    # Gérer les erreurs ici


Tout fonctionne bien ! Status Code: 200
{'poireau': {'data': {'poireau': 3}, 'name': 'poireau'}}


### Test avec les erreurs déclenchées dans l'API
Lancez le serveur dans votre terminal

```bash
flask --app example_4_errors_handlers.py run --debug
```

Timeout car fonction trop longue

In [14]:
TIMEOUT = 3
try:
    response = requests.get('http://localhost:5000/timeout3s', timeout=TIMEOUT)
    print(response.json())
except requests.exceptions.Timeout:  # changer en HttpError pour montrer que l'erreur n'est pas captée et donc casse le code
    print(f"Temps d'attente maximal dépassé {TIMEOUT}s")

Temps d'attente maximal dépassé 3s


Tentative d'accès à une route inexistante

In [15]:
# V1: erreur retournée dans le code de statut et dans le corps de la réponse
response = requests.get('http://localhost:5000/lien_qui_n_existe_pas')
print(response)
print('\n', response.text)

<Response [404]>

 <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In [16]:
# V2: même chose mais avec levée de l'erreur (raise_for_status)
# Et affichage du message automatique d'erreur (HTTPError)
try:
    response = requests.get('http://localhost:5000/lien_qui_n_existe_pas')
    response.raise_for_status()
    print("ceci ne sera pas lu ")
except requests.exceptions.HTTPError as erreur_levee_par_raise_for_status:
    print(erreur_levee_par_raise_for_status)

404 Client Error: NOT FOUND for url: http://localhost:5000/lien_qui_n_existe_pas


In [17]:
# V3: même chose mais avec levée de l'erreur et personnalisation du message d'erreur
try:
    response = requests.get('http://localhost:5000/lien_qui_n_existe_pas')
    response.raise_for_status()
    print("ceci ne sera pas lu ")
except requests.exceptions.HTTPError as erreur_levee_par_raise_for_status:
    print(f"|| 404 jupy ||")

|| 404 jupy ||


Accès aux routes qui causeront des erreurs de différentes manières

In [18]:
# La plus fatale des erreurs est une erreur de code qui n'est pas gérée
# Comme c'est le cas ici, cela va stopper le serveur
response = requests.get('http://localhost:5000/exception')
print(response)
print(response.text)

<Response [500]>
<!doctype html>
<html lang=en>
  <head>
    <title>Exception: || Exception ||
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "aOggdn3s2mbjSa7RxrJ6";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>Exception</h1>
<div class="detail">
  <p class="errormsg">Exception: || Exception ||
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-4401407328">
  <h4>File <cite class="filename">"/Users/alexisbogroff/Installs/miniforge3/envs/api/lib/python3.11/site-package

Systématisation des tests d'erreurs

In [19]:
# Nous allons à présent conserver ce format pour requêter notre API
# Autant en faire une fonction pour gangner en clarté
def get_with_error(uri, domain='http://localhost:5000/'):
    try:
        response = requests.get(domain + uri)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as e:
        print(f"|| {uri} jupy || -> {e}\n")
        print(response.status_code)
        print(response.text)

In [20]:
# type_error captée par le errorhandler
get_with_error('type_error/1')

|| type_error/1 jupy || -> 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:5000/type_error/1

500
|| Type Error ||


In [22]:
# 401 déclenchée via abort (la fonction n'a pas pu être exécutée au delà de abort)
get_with_error('401')

|| 401 jupy || -> 401 Client Error: UNAUTHORIZED for url: http://localhost:5000/401

401
Pas de chance vous tombez sur une erreur : 401 Unauthorized: || 401  recup||


In [23]:
# 402: erreur retournée dans le code de statut et dans le corps de la réponse
# Mais cette fois-ci, nous avons personnalisé le message d'erreur
# Contrairement à la tentative avec le lien_qui_n_existe_pas
get_with_error('402')

|| 402 jupy || -> 404 Client Error: NOT FOUND for url: http://localhost:5000/402

404
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In [24]:
# 403: Le message d'erreur non personnalisé s'affichera
# dans les logs et sera transmis dans le corps de la réponse
get_with_error('403')

|| 403 jupy || -> 403 Client Error: FORBIDDEN for url: http://localhost:5000/403

403
<!doctype html>
<html lang=en>
<title>403 Forbidden</title>
<h1>Forbidden</h1>
<p>|| 403 ||</p>



In [25]:
# 405: Le message d'erreur s'affichera dans le fichier de logs
get_with_error('405')

|| 405 jupy || -> 405 Client Error: METHOD NOT ALLOWED for url: http://localhost:5000/405

405
|| Tout va bien, nous gérons le problème :) ||
